# Part 3: Experiment and optimize the RAG application

- GitHub repository: https://github.com/anyscale/ray-summit-2023-training/tree/main
- Anyscale Endpoints: https://endpoints.anyscale.com/
- Ray documentation: https://docs.ray.io/
- LlamaIndex documentation: https://gpt-index.readthedocs.io/en/stable/

<span style="background: yellow; color: red; font-size: 1rem;"><b>TODO:</b></span> Add introduction paragraph for this tutorial.

<span style="background: yellow; color: red; font-size: 1rem;"><b>TODO:</b></span> assume evaluation functions have all been defined in previous part.

In [ ]:
import os

os.environ["ANYSCALE_API_BASE"] = "https://ray-summit-training-jrvwy.cld-kvedzwag2qa8i5bj.s.anyscaleuserdata-staging.com/v1/chat/completions"
os.environ["ANYSCALE_API_KEY"] = "tZLmCV1WtQAtnDx93MM5w3xaNhYV3whhcFRTKoH1GYQ"

os.environ["OPENAI_API_BASE"] = "https://api.openai.com/v1"
# os.environ["OPENAI_API_KEY"] = ...

## Strategy 1: Search for optimal configuration for standard components

<span style="background: yellow; color: red; font-size: 1rem;"><b>TODO:</b></span> add component parameter tuning experiments.

## Strategy 2: Use different data representation for retrieval vs. generation

The best data representation for retrieval isn't always the best for generation.

Often, it's best to embed the knowledge corpus in very small chunk sizes for the purpose of retrieval. This is because we aren't cramming a lot of textual information into a single vector, so each vector can retain more semantic information, without having to "average out" the information.

On the other hand, to generate high quality responses, we often need broader context around the specific information that we are querying for. For example, after retrieving a relevant code snippet, we also want to provide the surrounding documentation and discussion as context for a high quality response.

In this section, we show how a strategy where we use small, sentence level chunks for retrieval and a surrounding window as context for generation can help improve both retrieval and generation quality.

### Experiment configs

In [ ]:
BASE_MODEL = 'thenlper/gte-base'  
WINDOW_SIZE = 3
TOP_K = 20

TEST_SUBSAMPLE_N = 10

EXPERIMENT_NAME = 'sentence_window'

### Load data

In [ ]:
from pathlib import Path

DATASETS_DIRECTORY = Path("/efs/shared_storage/simon/datasets")

In [ ]:
import json
from llama_index.schema import Document

def read_json(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

def to_doc(entry_dict):
    return Document(text=entry_dict['text'], metadata={'source': entry_dict['source']}) 

def load_corpus(filename):
    sections = read_json(filename)
    docs = [to_doc(dict_) for dict_ in sections]
    return docs

In [ ]:
docs = load_corpus(DATASETS_DIRECTORY / 'eval_full_corpus.json')

### Build index and configure query engine

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index import ServiceContext, set_global_service_context, VectorStoreIndex
from llama_index.llms import OpenAI
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor

# create the sentence window node parser
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=WINDOW_SIZE,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

service_context = ServiceContext.from_defaults(
    embed_model=HuggingFaceEmbeddings(
        model_name=BASE_MODEL
    ),
    node_parser=node_parser,
)

index = VectorStoreIndex.from_documents(
    docs, 
    service_context=service_context, 
    show_progress=True
)

In [ ]:
retriever = index.as_retriever(similarity_top_k=TOP_K)

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=TOP_K,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

### Run evaluation

In [ ]:
import json
import numpy as np

from eval import evaluate_e2e, evaluate_retrieval

#### Load data

In [ ]:
with open("golden-responses.json", "r") as file:
    golden_dataset = json.load(file)

In [ ]:
golden_dataset[0]

In [ ]:
queries = [item['question'] for item in golden_dataset]
golden_response = [item['response'] for item in golden_dataset]
golden_sources = [item['source'] for item in golden_dataset]

#### Retrieval-only

In [ ]:
results = evaluate_retrieval(retriever, queries, golden_sources)

In [ ]:
hits = [r['is_hit'] for r in results]
hits

In [ ]:
np.mean(hits)

#### End-to-end

In [ ]:
results = evaluate_e2e(query_engine, queries, golden_response, verbose=True)

In [ ]:
scores = [r.score for r in results]
scores

In [ ]:
np.mean(scores)

## Strategy 3: Fine-tune embeddings

In this section, we explore fine-tuning embedding model to improve retrieval performance.
We consider the "cold start" regime, where we haven't deployed the model, and haven't collected any user queries or labeled "golden" context. 

Therefore, we consider a synthetic data approach, where we leverage LLM to generation question, relevant context, answer pairs from our knowledge corpus.

### Experiment configs

In [ ]:
import os

# Since we are working with a relatively large set of documents, 
# we sub-sample the data for quicker iterations.
SUBSAMPLE_RATIO = 0.05

# can be any sentence-transformer compatible model
# https://www.sbert.net/docs/pretrained_models.html
BASE_MODEL = 'BAAI/bge-small-en'  

# Select a chunk size that is: 
# 1) under the context window limit of the chosen model, and
# 2) close to the desired chunk size at retrieval time
FINETUNE_CHUNK_SIZE = 512


ROOT_DIR = Path(os.getcwd()).parent
print(ROOT_DIR)

### Load data

We start by loading our knowledge corpus (i.e. the ray documentation webpages that we've downloaded and parsed previously).

In [ ]:
from pathlib import Path

DATASETS_DIRECTORY = Path("/efs/shared_storage/simon/datasets")

In [ ]:
import json
from llama_index.schema import Document

def read_json(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

def to_doc(entry_dict):
    return Document(text=entry_dict['text'], metadata={'source': entry_dict['source']}) 

def load_corpus(filename):
    sections = read_json(filename)
    docs = [to_doc(dict_) for dict_ in sections]
    return docs

In [ ]:
docs = load_corpus(DATASETS_DIRECTORY / 'eval_full_corpus.json')

Now, we split the documents into chunks. The main considerations on the chunk size here are:
1. need to fit into the context window of the embedding model that we want to finetune (512 for the sentence transformer model that we selected, similar to most open source models.)
2. should be close to the chunk size we want to use at retrieval time (so it's best to run some experiments to determine the best chunk size for your application first, see strategy 1). 

In [ ]:
from llama_index.node_parser import SimpleNodeParser

parser = SimpleNodeParser.from_defaults(chunk_size=FINETUNE_CHUNK_SIZE)
nodes = parser.get_nodes_from_documents(docs, show_progress=True)
print('Parsed {} docs into {} nodes'.format(len(docs), len(nodes)))

### Subsample and create train/validation split

In [ ]:
from utils import train_test_split, subsample

In [ ]:
train_nodes, val_nodes = train_test_split(nodes)
print('Split dataset into: {} train nodes, {} val nodes'.format(len(train_nodes), len(val_nodes)))

In [ ]:
train_nodes = subsample(train_nodes, SUBSAMPLE_RATIO)
val_nodes = subsample(val_nodes, SUBSAMPLE_RATIO)
print('Subsampled dataset into: {} train nodes, {} val nodes'.format(len(train_nodes), len(val_nodes)))

### Generate synthetic dataset

Now, we will generate a synthetic dataset of question, "golden" context, and "golden" answer dataset by leveraing an LLM (by default using gpt-3.5-turbo from OpenAI)

In [ ]:
from llama_index.finetuning import generate_qa_embedding_pairs

train_dataset = generate_qa_embedding_pairs(train_nodes)
val_dataset = generate_qa_embedding_pairs(val_nodes)

Let's save the dataset for future use since it's fairly time consuming to generate.

In [ ]:
train_dataset.save_json(Path(ROOT_DIR, "finetune/synthetic_train_dataset.json"))
val_dataset.save_json(Path(ROOT_DIR, "finetune/synthetic_val_dataset.json"))

### Run embedding finetuning

Now, we are ready to fine-tune our embedding model!

In [ ]:
from llama_index.finetuning import EmbeddingQAFinetuneDataset

In [ ]:
train_dataset = EmbeddingQAFinetuneDataset.from_json(Path(ROOT_DIR, "finetune/synthetic_train_dataset.json"))
val_dataset = EmbeddingQAFinetuneDataset.from_json(Path(ROOT_DIR, "finetune/synthetic_val_dataset.json"))

We can construct a fine-tune engine, which is an easy to use interface for running fine-tuning jobs (either locally or via a API-based service).

Here, we use the sentence transformer fine-tuning engine to run fine-tuning locally on the ray cluster.

In [ ]:
from llama_index.finetuning import SentenceTransformersFinetuneEngine

finetune_engine = SentenceTransformersFinetuneEngine(
    train_dataset,
    model_id="thenlper/gte-base",
    model_output_path="../finetune/exp_finetune_test",
    val_dataset=val_dataset,
    epochs=1,
)

For demonstration purpose, we will run the fine-tuning job for 2 epoches over our synthetic dataset.
In practice, you should use the validation loss to determine how many epochs to fine-tune the embedding model for.

In [ ]:
finetune_engine.finetune()

After the fine-tuning job finishes, we can easy get a referene to the fine-tuned model to be used in our LlamaIndex application.

In [ ]:
embed_model = finetune_engine.get_finetuned_model()

### Evaluate our fine-tuned embedding model

Now, we will leverage the retrieval evaluation process we built out in [part 2](www.google.com) to assess the quality of our fine-tuned embedding model.

In [ ]:
import re
import json
from pathlib import Path

# Load labeled eval dataset
golden_dataset_path = Path("../datasets/eval-dataset-v1.jsonl")

with open(golden_dataset_path, "r") as f:
    test_dataset = [json.loads(item) for item in list(f)]
    
len(test_dataset)

In [ ]:
test_dataset[:5]

In [ ]:
queries = [item['question'] for item in test_dataset]
golden_sources = [item['source'] for item in test_dataset]

Now, we can build index with our fine-tuned embedding.

In [ ]:
from llama_index import VectorStoreIndex, ServiceContext

In [ ]:
service_context = ServiceContext.from_defaults(
    embed_model=embed_model,
    chunk_size=512,
)

index = VectorStoreIndex.from_documents(
    docs, 
    service_context=service_context, 
    show_progress=True
)

In [ ]:
retriever = index.as_retriever(similarity_top_k=5)

### Run retrieval evaluation 

In [ ]:
from eval import evaluate_retrieval

results = evaluate_retrieval(retriever, queries, golden_sources)

In [ ]:
total_hits = sum(result["is_hit"] for result in results)
hit_percentage = total_hits / len(results)
hit_percentage

In [ ]:
average_score = sum(result["score"] for result in results) / len(results)
average_score